# Download documents

In [1]:
urls = [
    "https://bocyl.jcyl.es/boletines/2024/01/02/xml/BOCYL-D-02012024-12.xml",
    "https://bocyl.jcyl.es/boletines/2023/07/18/xml/BOCYL-D-18072023-10.xml",
]

In [2]:
from pathlib import Path
folder_documents = Path("/workspace/data/documents/BOCYL")

In [3]:
from modules.preprocessing import BOCYLMarkdownExporter
exporter = BOCYLMarkdownExporter(folder_documents)

In [4]:
paths = []

for url in urls:
    exporter.export(url)
    filename = url.split('/')[-1].split('.')[0]
    path = folder_documents / f"{filename}.md"
    paths.append(path)

paths

ConnectionError: HTTPSConnectionPool(host='bocyl.jcyl.es', port=443): Max retries exceeded with url: /boletines/2024/01/02/xml/BOCYL-D-02012024-12.xml (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0xffff9cd5e030>: Failed to resolve 'bocyl.jcyl.es' ([Errno -3] Temporary failure in name resolution)"))

## Create vector db

### Instanciate client

In [5]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from pathlib import Path
import os

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Define the path where the database will be stored
db_path = "/workspace/data/vectordb/chromadb"

# Create directory if it doesn't exist
os.makedirs(db_path, exist_ok=True)

# Initialize the ChromaDB client with persistent storage
client = chromadb.PersistentClient(path=db_path)

### Create collection

In [7]:
# Create a new collection (or get existing one)
collection_name = "bocyl"
collection = client.get_or_create_collection(name=collection_name)

### Insert documents

#### Define functions to extract metadata

In [8]:
import re
from datetime import datetime

# Function to extract metadata from filename
def extract_metadata_from_filename(filename):
    # Pattern: BOCYL-D-DDMMYYYY-NN
    pattern = r"BOCYL-D-(\d{2})(\d{2})(\d{4})-(\d+)"
    match = re.match(pattern, filename)
    
    if match:
        day, month, year, doc_num = match.groups()
        # Create date object
        doc_date = datetime.strptime(f"{day}/{month}/{year}", "%d/%m/%Y").strftime("%Y-%m-%d")
        
        return {
            "doc_id": filename,
            "date": doc_date,
            "doc_number": int(doc_num),
            "source": "BOCYL",
            "type": "official_document"
        }
    return {"doc_id": filename}

# Function to extract potential metadata from document content
def extract_metadata_from_content(content):
    # Example patterns to look for in document content
    title_match = re.search(r"^# (.+)$", content, re.MULTILINE)
    title = title_match.group(1) if title_match else "Unknown Title"
    
    # Could extract more metadata based on document structure
    # This is a simplified example
    
    return {
        "title": title
    }

#### Define embedding model

In [9]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-small-en-v1.5',
    model_kwargs={'device': device}
)

#### Upsert documents

In [10]:
for path in paths:
    # Read document content
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    
    # Extract document ID and filename
    doc_id = path.stem
    
    # Check if document already exists (using the first chunk's ID as indicator)
    first_chunk_id = f"{doc_id}_0"
    existing = collection.get(ids=[first_chunk_id])
    
    if len(existing['ids']) > 0:
        print(f"Document {doc_id} already exists. Deleting existing chunks...")
        # Get all chunks for this document
        doc_chunks = collection.get(where={"doc_id": doc_id})
        if doc_chunks and len(doc_chunks['ids']) > 0:
            # Delete all existing chunks for this document
            collection.delete(ids=doc_chunks['ids'])
    
    # Extract metadata from filename
    filename_metadata = extract_metadata_from_filename(doc_id)
    
    # Extract metadata from content
    content_metadata = extract_metadata_from_content(content)
    
    # Combine metadata
    document_metadata = {**filename_metadata, **content_metadata}
    
    # Add doc_id to metadata
    document_metadata["doc_id"] = doc_id
    
    # Split document into chunks (simple approach)
    chunk_size = 1000
    chunks = [content[i:i+chunk_size] for i in range(0, len(content), chunk_size)]
    
    # Process each chunk
    ids = []
    metadatas = []
    documents = []
    
    for i, chunk in enumerate(chunks):
        chunk_id = f"{doc_id}_{i}"
        ids.append(chunk_id)
        
        # Add chunk-specific metadata
        chunk_metadata = {
            **document_metadata,
            "chunk_id": i,
            "chunk_total": len(chunks)
        }
        metadatas.append(chunk_metadata)
        documents.append(chunk)
    
    # Instead of encoding each chunk individually in a loop
    embeddings = model.embed_documents(chunks)
    
    # Add to collection
    collection.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=metadatas,
        documents=chunks
    )
    
    print(f"Added {doc_id} with {len(chunks)} chunks and metadata: {document_metadata}")

Document BOCYL-D-02012024-12 already exists. Deleting existing chunks...
Added BOCYL-D-02012024-12 with 104 chunks and metadata: {'doc_id': 'BOCYL-D-02012024-12', 'date': '2024-01-02', 'doc_number': 12, 'source': 'BOCYL', 'type': 'official_document', 'title': 'ORDEN AGR/1488/2023, de 22 de diciembre, por la que se determinan en Castilla y León las obligaciones de la condicionalidad reforzada y el procedimiento para las penalizaciones por incumplimiento de la condicionalidad social, que han de cumplir las personas beneficiarias de ayudas de la Política Agraria Común que perciban pagos directos y determinados pagos anuales por superficies y animales para el desarrollo rural, en el marco del Plan Estratégico Nacional de la PAC 2023-2027 de España.'}
Added BOCYL-D-18072023-10 with 121 chunks and metadata: {'doc_id': 'BOCYL-D-18072023-10', 'date': '2023-07-18', 'doc_number': 10, 'source': 'BOCYL', 'type': 'official_document', 'title': 'ORDEN EYH/886/2023, de 11 de julio, por la que se aprue

### Count documents inserted

In [11]:
# Count items in collection
count = collection.count()
print(f"Total number of chunks in collection: {count}")

# Get unique document IDs
results = collection.get(
    where={"chunk_id": 0}  # Get only first chunk of each document
)
print(f"Number of documents: {len(results['ids'])}")

Total number of chunks in collection: 225
Number of documents: 2


## Query the vector db

### Define query

In [12]:
query = "¿Qué medidas de condicionalidad reforzada se mencionan en la normativa agraria?"

### Encode query with embedding model

In [13]:
query_embedding = model.embed_query(query)

AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'encode'

### Run the query

In [ ]:
# Run the query - include_distances=True is key for getting scores
search_results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
)

# Display results with scores
for i, (id, document, metadata, distance) in enumerate(zip(
        search_results["ids"][0],
        search_results["documents"][0],
        search_results["metadatas"][0],
        search_results["distances"][0]  # This contains the similarity scores
    )):
    # Convert distance to similarity score (closer to 1 is better)
    # ChromaDB typically returns distances, where smaller is better
    # For cosine similarity, you can convert: similarity = 1 - distance
    similarity_score = 1 - distance  # For cosine distance
    
    print(f"Result {i+1}:")
    print(f"  Document: {metadata['doc_id']}")
    print(f"  Chunk: {metadata['chunk_id']}")
    print(f"  Similarity Score: {similarity_score:.4f}")
    print(f"  Text: {document[:150]}...")
    print()

Result 1:
  Document: BOCYL-D-02012024-12
  Chunk: 4
  Similarity Score: 0.6451
  Text: as competentes por razón de las materias relacionadas con la condicionalidad reforzada, que dispongan de datos e información relacionados con las obli...

Result 2:
  Document: BOCYL-D-02012024-12
  Chunk: 10
  Similarity Score: 0.6417
  Text: to de aplicación.

Esta norma será de aplicación a todas las personas beneficiarias de los regímenes de ayudas recogidos en el ANEXO I de la presente ...

Result 3:
  Document: BOCYL-D-02012024-12
  Chunk: 11
  Similarity Score: 0.5544
  Text: . Relación de requisitos legales de gestión y de las buenas condiciones agrarias y medioambientales de la condicionalidad reforzada.

Las personas ben...



## Refactor the code

In [22]:
from modules.vectordb import VectorDBQuerier

In [23]:
# Initialize the querier with your collection and model
querier = VectorDBQuerier(collection, model)
querier

## Multiple queries

In [25]:
# Run a single query
result1 = querier.query(
    "¿Qué medidas de condicionalidad reforzada se mencionan en la normativa agraria?",
    n_results=3
)

Result 1:
  Document: BOCYL-D-02012024-12
  Chunk: 4
  Similarity Score: 0.6451
  Text: as competentes por razón de las materias relacionadas con la condicionalidad reforzada, que dispongan de datos e información relacionados con las obli...

Result 2:
  Document: BOCYL-D-02012024-12
  Chunk: 10
  Similarity Score: 0.6417
  Text: to de aplicación.

Esta norma será de aplicación a todas las personas beneficiarias de los regímenes de ayudas recogidos en el ANEXO I de la presente ...

Result 3:
  Document: BOCYL-D-02012024-12
  Chunk: 11
  Similarity Score: 0.5544
  Text: . Relación de requisitos legales de gestión y de las buenas condiciones agrarias y medioambientales de la condicionalidad reforzada.

Las personas ben...



In [26]:
result1

{'query': '¿Qué medidas de condicionalidad reforzada se mencionan en la normativa agraria?',
 'results': [{'document_id': 'BOCYL-D-02012024-12',
   'chunk_id': 4,
   'metadata': {'title': 'ORDEN AGR/1488/2023, de 22 de diciembre, por la que se determinan en Castilla y León las obligaciones de la condicionalidad reforzada y el procedimiento para las penalizaciones por incumplimiento de la condicionalidad social, que han de cumplir las personas beneficiarias de ayudas de la Política Agraria Común que perciban pagos directos y determinados pagos anuales por superficies y animales para el desarrollo rural, en el marco del Plan Estratégico Nacional de la PAC 2023-2027 de España.',
    'date': '2024-01-02',
    'type': 'official_document',
    'chunk_id': 4,
    'chunk_total': 104,
    'source': 'BOCYL',
    'doc_id': 'BOCYL-D-02012024-12',
    'doc_number': 12},
   'text': 'as competentes por razón de las materias relacionadas con la condicionalidad reforzada, que dispongan de datos e infor

In [28]:
result3 = querier.query(
    "Requisitos para beneficiarios de fondos FEAGA",
    n_results=5
)

Result 1:
  Document: BOCYL-D-18072023-10
  Chunk: 119
  Similarity Score: 0.5955
  Text:  cuenta los plazos fijados respectivamente para el mantenimiento del empleo y para el mantenimiento de la inversión. Esta responsabilidad deberá ser a...

Result 2:
  Document: BOCYL-D-18072023-10
  Chunk: 33
  Similarity Score: 0.5909
  Text: iterios de sostenibilidad establecidos en el artículo 29 de la Directiva (UE) 2018/2001 del Parlamento Europeo y del Consejo.

Base cuarta.  Entidades...

Result 3:
  Document: BOCYL-D-18072023-10
  Chunk: 75
  Similarity Score: 0.5840
  Text: haga constar la fecha y el órgano o dependencia en que fueron presentados o, en su caso, emitidos, y no hayan transcurrido más de cinco años desde la ...

Result 4:
  Document: BOCYL-D-02012024-12
  Chunk: 2
  Similarity Score: 0.5748
  Text:  en relación con la ayuda a los planes estratégicos que deben elaborar los Estados miembros en el marco de la política agrícola común (PAC) financiada...

Result 5:
  Document: BO

## Refactor vectordb instance

In [ ]:
from modules.vectordb import get_vectordb_bocyl